In [ ]:
%cd '/content/drive/My Drive/Toxic Content Detection'

/content/drive/My Drive/Toxic Content Detection


In [ ]:
import warnings
warnings.simplefilter("ignore")

from time import time
import pandas as pd
import numpy as np

import pickle

In [ ]:
import re
import string

def preprocess(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = re.split('\W+', text)
    text = " ".join(tokens).strip().lower()
    return text

In [ ]:
train = pd.read_csv('Datasets/jigsaw-toxic-comment/train.csv', index_col="id")
train['toxic'] = (train['toxic'] | train['severe_toxic'] | train['obscene'] | train['threat'] | train['insult'] | train['identity_hate']).astype('category')
train = train.drop(['severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1)
train['comment_text'] = train['comment_text'].apply(preprocess)

train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159571 entries, 0000997932d777bf to fff46fc426af1f9a
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   comment_text  159571 non-null  object  
 1   toxic         159571 non-null  category
dtypes: category(1), object(1)
memory usage: 2.6+ MB


In [ ]:
test = pd.read_csv('Datasets/jigsaw-toxic-comment/test.csv', index_col="id")
test_labels = pd.read_csv('Datasets/jigsaw-toxic-comment/test_labels.csv', index_col="id")
test = test.join(test_labels)
test['toxic'] = (test['toxic'] | test['severe_toxic'] | test['obscene'] | test['threat'] | test['insult'] | test['identity_hate']).astype('category')
test = test.drop(['severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1)
test = test.drop(test[test['toxic'] == -1].index)
test['comment_text'] = test['comment_text'].apply(preprocess)

test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63978 entries, 0001ea8717f6de06 to fffb5451268fb5ba
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   comment_text  63978 non-null  object  
 1   toxic         63978 non-null  category
dtypes: category(1), object(1)
memory usage: 1.0+ MB


In [ ]:
x_test_raw, y_test = test['comment_text'], test['toxic']
x_train_raw, y_train = train['comment_text'], train['toxic']

In [ ]:
!pip3 install --quiet sentencepiece

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import sentencepiece as spm
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_hub as hub

def use_lite_embed():
    module = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-lite/2")
    input_placeholder = tf.sparse_placeholder(tf.int64, shape=[None, None])
    encodings = module(
        inputs=dict(
            values=input_placeholder.values,
            indices=input_placeholder.indices,
            dense_shape=input_placeholder.dense_shape))
    
    with tf.Session() as sess:
        spm_path = sess.run(module(signature="spm_path"))
    sp = spm.SentencePieceProcessor()
    sp.Load(spm_path)

    def process_to_IDs_in_sparse_format(sp, sentences):
        ids = [sp.EncodeAsIds(x) for x in sentences]
        max_len = max(len(x) for x in ids)
        dense_shape=(len(ids), max_len)
        values=[item for sublist in ids for item in sublist]
        indices=[[row,col] for row in range(len(ids)) for col in range(len(ids[row]))]
        return (values, indices, dense_shape)

    def embed(msgs):
        values, indices, dense_shape = process_to_IDs_in_sparse_format(sp, msgs)
        
        with tf.Session() as session:
            session.run([tf.global_variables_initializer(), tf.tables_initializer()])
            return session.run(
                encodings,
                feed_dict={input_placeholder.values: values,
                            input_placeholder.indices: indices,
                            input_placeholder.dense_shape: dense_shape})
    return embed

embed = use_lite_embed()

     |████████████████████████████████| 1.0MB 9.0MB/s 
Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
def batch_map(func, iterable, batch_size=1):
    from tqdm import tqdm
    l = len(iterable)
    result = list()
    for ndx in tqdm(range(0, l, batch_size)):
        result.extend(func(iterable[ndx:min(ndx + batch_size, l)]))
    return result

In [ ]:
print("Embedding train set...", end="")
x_train = batch_map(embed, x_train_raw.to_numpy().reshape((-1)), batch_size=10000)
print("Done")

print("Embedding test set...", end="")
x_test = batch_map(embed, x_test_raw.to_numpy().reshape((-1)), batch_size=10000)
print("Done")

In [ ]:
np.savez('Models/Embed+SKLearn/Original/data.npz', x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier, PassiveAggressiveClassifier

classifiers = [("LR", LogisticRegression()),
               ("SGD", SGDClassifier()),
               ("Ridge", RidgeClassifier()),
               ("LDA", LinearDiscriminantAnalysis()),
               ("PA", PassiveAggressiveClassifier())]

from sklearn.ensemble import VotingClassifier
clf = VotingClassifier(estimators=classifiers)

In [ ]:
# Train with timing
print("Training...", end="")
time_to_train = time()
clf.fit(x_train, y_train)
time_to_train = time() - time_to_train
print(f"Done. Took {time_to_train}s")

# Save model
print("Saving model...", end="")
with open(f"Models/Embed+SKLearn/Original/USEL-Voting.model", 'wb') as clf_file:
    pickle.dump(clf, clf_file)
print("Done")

# Predict with timing
print("Predicting for test set...", end="")
time_to_predict = time()
prediction = clf.predict(x_test)
time_to_predict = time() - time_to_predict
print(f"Done. Took {time_to_predict}s")

Training...Done. Took 34.23701071739197s
Saving model...Done
Predicting for test set...Done. Took 1.3672964572906494s


In [ ]:
# Metrics
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

def confusion_matrix_proper(y_true, pred):
    return ",".join(str(score) for score in confusion_matrix(y_true, pred).ravel())

metrics = [
           {"name": "accuracy", "metric": accuracy_score},
           {"name": "balanced_accuracy", "metric": balanced_accuracy_score},
           {"name": "tn,fp,fn,tp", "metric": confusion_matrix_proper},
           {"name": "precision", "metric": precision_score},
           {"name": "recall", "metric": recall_score},
           {"name": "f1", "metric": f1_score},
]

In [ ]:
print(f"Metrics:")
with open('Models/Embed+SKLearn/Original/models.csv', 'a') as logfile:
    logfile.write(f"{time()},Voting,USEL,{time_to_train},{time_to_predict}")
    for metric in metrics:
        score = metric['metric'](y_test, prediction)
        print(f"{metric['name']}: {score}")
        logfile.write(f",{score}")
    logfile.write("\n")


Metrics:
accuracy: 0.9300384507174341
balanced_accuracy: 0.8082348192563356
tn,fp,fn,tp: 55401,2334,2142,4101
precision: 0.6372960372960373
recall: 0.6568957232099952
f1: 0.6469474680548983
